# Preproccessing Data

### Import Dataset

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from nltk.tokenize import word_tokenize
import re
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string

In [2]:
df = pd.read_csv("reviews.csv")

df.head(5)

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [3]:
df.Rating.value_counts()

Rating
5    22095
1    17653
4     7842
2     7118
3     6886
Name: count, dtype: int64

### Feature Selection

In [5]:
df.drop(columns=["Time_submitted", "Total_thumbsup", "Reply"], inplace=True)

df.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1


In [8]:
df = df[(df.Rating == 1) | (df.Rating == 5)]

df.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1
6,I love the selection and the lyrics are provid...,5


In [10]:
labels = [0 if rate == 1 else 1 for rate in df.Rating]
df["Labels"] = labels

df.head()

,Review,Rating,Labels
0,"Great music service, the audio is high quality...",5,1
1,Please ignore previous negative rating. This a...,5,1
3,Really buggy and terrible to use as of recently,1,0
4,Dear Spotify why do I get songs that I didn't ...,1,0
6,I love the selection and the lyrics are provid...,5,1


In [16]:
df[df.Labels == 0].iloc[:3000].sample(frac=1, random_state=42).reset_index(drop=True)

,Review,Rating,Labels
0,Its autoplay is off yet playing as autoplay.,1,0
1,This is rubbish can't even listen the song u c...,1,0
2,I don't like it u can't pick what playlist you...,1,0
3,I've used spotify for years and have payed for...,1,0
4,I'm giving one star ⭐ because the application ...,1,0
...,...,...,...
2995,Very very ads I request all my friends please ...,1,0
2996,Very nice i love spotify,1,0
2997,Last update ruined playback. Pocast keeps stop...,1,0
2998,Cannot use the app. When i want to listen to a...,1,0


### Duplicated Data

### Missing Value

### Case Folding

### Tokenized

### Stopwords

### Stemming

# Pembobotan Kata

### Frequency of Word

### Term Frequncy

### Inverse Document Frequency

### TF-IDF

# Modeling dan Testing

### Pembuatan class

### Training

### Testing

### Conclusion